In [ ]:
import getpass
import numpy as np
from math import pi
import matplotlib.pyplot as plt
import ai.fsg.golfproc as golfproc
import ai.fsg.runtimeinfo as runtimeinfo
import ai.fsg.fsg_plot as fsg_plot
import ai.fsg.golfproc as fsg_proc
import ai.fsg.ai_radar_fft as af
# import progs.test_trigger_mode as tt
from ai.fsg.fsg_ball_club_tracks import  Track_Trigger_Mode
import ai.fsg.aoaproc as aoaproc
import os
import csv

import mplcursors

USER = getpass.getuser()

In [ ]:
plt.rcParams['figure.figsize'] = [8, 5]
%matplotlib notebook

# Multiple File Analysis to CSV

In [ ]:
#select folder with known data. Script will determine number of raw data files and loop through files and output to
#csv. Data types that are saved are magnitude, noise, snr, and aoa. Results are pushed to (data)_results.csv.

data_dir = "/home/"+ USER +"/data/fsg/golf_data/rev3p2p2/angle_of_arrival/" \
             "20200622/june_build/u6/"

mag_array_ch1 = []
mag_array_ch2 = []
mag_array_ch3 = []
mag_array_ch4 = []
noise_array_ch1 = []
noise_array_ch2 = []
noise_array_ch3 = []
noise_array_ch4 = []
snr_array_ch1 = []
snr_array_ch2 = []
snr_array_ch3 = []
snr_array_ch4 = []

csv_mag = "mag_results.csv"
csv_noise = "noise_results.csv"
csv_snr = "snr_results.csv" 
csv_aoa = "aoa_results.csv" 
aoa_fields = ["aoa_diffphase_el", "aoa_diffphase_az","mean_el_trigger", "el_angle_trigger_std", "mean_el_flight", \
              "el_angle_flight_std", "mean_az_trigger", "az_angle_trigger_std", "mean_az_flight", "az_angle_flight_std"]

file_num = 0
# for path in pathlib.Path(data_dir).iterdir():
while True:
    if os.path.isfile(data_dir +"r_Test"+ str(file_num) + ".bin") == 1:
        file_num += 1 
    else:
        break
print(file_num)
        
for x in range(0,file_num):
    data_samp = "Test" + str(x)
    # filename = data_samp + ".BIN"
    fft_filename = "fflt_" + data_samp + ".bin"
    raw_filename = "r_" + data_samp + ".bin"

    #tee_pos = [1.448, 0.0, 0.1] #driver_pga0_run2/TEST_10.BIN
    #tee_pos = [3.0, 0.0, 0.1]
    #MAKE SURE YOU CHANGE TEE_POS Z (HEIGHT) FOR ELEVATED LM DATA!!
    #tee_pos = [3.0, 0.0, -1]
    # tee_pos = [2.5, 0.5, 2.0]

    proc = golfproc.GolfProcessing(data_dir = data_dir, \
                               debug_log = 0, debug_plot = 2)

    ##need to choose/set which data file type you have set as 'filename'
    proc.set_system_snapshot(data_dir, pb_filename=None,
    #                         config_filename="../../../../python/config.json",
                         config_filename=data_dir + "/config.json",

                         myconfig_filename="../../../../myconfig.json",
                         fft_filename=fft_filename, waveform_filename=raw_filename) ##uncomment line to use fft data file
    #                         fft_filename=None, waveform_filename=filename) ##uncomment line to use raw data file

    # ri = runtimeinfo.FpgaRuntimeInfo()
    # ri.club_type = 1  # 0=>"Driver", 1=>"5 Iron" currently
    #ri.tee_pos_z = -0.1

    ##PICK ONE and uncomment 
    ##without changing default bins
    aoa_array = proc.do_aoa_processing(0,0*pi/180,1000,15000)
    ##changing default bins
    #proc.do_aoa_processing(0,0,50,100)
    
    mag_array_ch1.insert(x, proc.beacon.magnitude_by_frame_dB[:,0])
    mag_array_ch2.insert(x, proc.beacon.magnitude_by_frame_dB[:,1])
    mag_array_ch3.insert(x, proc.beacon.magnitude_by_frame_dB[:,2])
    mag_array_ch4.insert(x, proc.beacon.magnitude_by_frame_dB[:,3])
    
    noise_array_ch1.insert(x, proc.beacon.noise_by_frame_dB[:,0])
    noise_array_ch2.insert(x, proc.beacon.noise_by_frame_dB[:,1])
    noise_array_ch3.insert(x, proc.beacon.noise_by_frame_dB[:,2])
    noise_array_ch4.insert(x, proc.beacon.noise_by_frame_dB[:,3])
    
    snr_array_ch1.insert(x, proc.beacon.snr_by_frame_dB[:,0])
    snr_array_ch2.insert(x, proc.beacon.snr_by_frame_dB[:,1])
    snr_array_ch3.insert(x, proc.beacon.snr_by_frame_dB[:,2])
    snr_array_ch4.insert(x, proc.beacon.snr_by_frame_dB[:,3])


  
with open(csv_mag, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["channel 1"])
    csvwriter.writerows(mag_array_ch1)
    csvwriter.writerow(["channel 2"])
    csvwriter.writerows(mag_array_ch2)
    csvwriter.writerow(["channel 3"])
    csvwriter.writerows(mag_array_ch3)
    csvwriter.writerow(["channel 4"])
    csvwriter.writerows(mag_array_ch4)
    csvfile.close()
    
with open(csv_noise, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["channel 1"])
    csvwriter.writerows(noise_array_ch1)
    csvwriter.writerow(["channel 2"])
    csvwriter.writerows(noise_array_ch2)
    csvwriter.writerow(["channel 3"])
    csvwriter.writerows(noise_array_ch3)
    csvwriter.writerow(["channel 4"])
    csvwriter.writerows(noise_array_ch4)
    csvfile.close()
    
with open(csv_snr, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["channel 1"])
    csvwriter.writerows(mag_array_ch1)
    csvwriter.writerow(["channel 2"])
    csvwriter.writerows(mag_array_ch2)
    csvwriter.writerow(["channel 3"])
    csvwriter.writerows(mag_array_ch3)
    csvwriter.writerow(["channel 4"])
    csvwriter.writerows(mag_array_ch4)
    csvfile.close()
    
with open(csv_aoa, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(aoa_fields)
    csvwriter.writerow(aoa_array[2:10])
    csvfile.close()

# Data Set Comparison

In [ ]:
#select folders with known data. Script will determine number of raw data files and loop through files and plot both
#files to compare results
#data set comparison one

data_dir1 = "/home/"+ USER +"/data/fsg/golf_data/rev3p2p2/angle_of_arrival/" \
             "20200619/june_build/u6/"

#data set comparison two
data_dir2 = "/home/"+ USER +"/data/fsg/golf_data/rev3p2p2/angle_of_arrival/" \
             "20200619/june_build/u5/"

set1_mag_array_ch1 = []
set1_mag_array_ch2 = []
set1_mag_array_ch3 = []
set1_mag_array_ch4 = []
set1_noise_array_ch1 = []
set1_noise_array_ch2 = []
set1_noise_array_ch3 = []
set1_noise_array_ch4 = []
set1_snr_array_ch1 = []
set1_snr_array_ch2 = []
set1_snr_array_ch3 = []
set1_snr_array_ch4 = []

set2_mag_array_ch1 = []
set2_mag_array_ch2 = []
set2_mag_array_ch3 = []
set2_mag_array_ch4 = []
set2_noise_array_ch1 = []
set2_noise_array_ch2 = []
set2_noise_array_ch3 = []
set2_noise_array_ch4 = []
set2_snr_array_ch1 = []
set2_snr_array_ch2 = []
set2_snr_array_ch3 = []
set2_snr_array_ch4 = []


file_num1 = 0
# for path in pathlib.Path(data_dir).iterdir():
while True:
    if os.path.isfile(data_dir1 +"r_Test"+ str(file_num1) + ".bin") == 1:
        file_num1 += 1 
    else:
        break
print(file_num1)

file_num2 = 0
# for path in pathlib.Path(data_dir).iterdir():
while True:
    if os.path.isfile(data_dir2 +"r_Test"+ str(file_num2) + ".bin") == 1:
        file_num2 += 1 
    else:
        break
print(file_num2)
        
for x in range(0,file_num1):
    data_samp = "Test" + str(x)
    # filename = data_samp + ".BIN"
    fft_filename = "fflt_" + data_samp + ".bin"
    raw_filename = "r_" + data_samp + ".bin"

    #tee_pos = [1.448, 0.0, 0.1] #driver_pga0_run2/TEST_10.BIN
    #tee_pos = [3.0, 0.0, 0.1]
    #MAKE SURE YOU CHANGE TEE_POS Z (HEIGHT) FOR ELEVATED LM DATA!!
    #tee_pos = [3.0, 0.0, -1]
    # tee_pos = [2.5, 0.5, 2.0]

    proc = golfproc.GolfProcessing(data_dir = data_dir1, \
                               debug_log = 0, debug_plot = 2)

    ##need to choose/set which data file type you have set as 'filename'
    proc.set_system_snapshot(data_dir1, pb_filename=None,
    #                         config_filename="../../../../python/config.json",
                         config_filename=data_dir1 + "/config.json",

                         myconfig_filename="../../../../myconfig.json",
                         fft_filename=fft_filename, waveform_filename=raw_filename) ##uncomment line to use fft data file
    #                         fft_filename=None, waveform_filename=filename) ##uncomment line to use raw data file

    # ri = runtimeinfo.FpgaRuntimeInfo()
    # ri.club_type = 1  # 0=>"Driver", 1=>"5 Iron" currently
    #ri.tee_pos_z = -0.1

    ##PICK ONE and uncomment 
    ##without changing default bins
    aoa_array = proc.do_aoa_processing(0,0*pi/180,1000,15000)
    ##changing default bins
    #proc.do_aoa_processing(0,0,50,100)
    
    set1_mag_array_ch1.insert(x, proc.beacon.magnitude_by_frame_dB[:,0])
    set1_mag_array_ch2.insert(x, proc.beacon.magnitude_by_frame_dB[:,1])
    set1_mag_array_ch3.insert(x, proc.beacon.magnitude_by_frame_dB[:,2])
    set1_mag_array_ch4.insert(x, proc.beacon.magnitude_by_frame_dB[:,3])
    
    set1_noise_array_ch1.insert(x, proc.beacon.noise_by_frame_dB[:,0])
    set1_noise_array_ch2.insert(x, proc.beacon.noise_by_frame_dB[:,1])
    set1_noise_array_ch3.insert(x, proc.beacon.noise_by_frame_dB[:,2])
    set1_noise_array_ch4.insert(x, proc.beacon.noise_by_frame_dB[:,3])
    
    set1_snr_array_ch1.insert(x, proc.beacon.snr_by_frame_dB[:,0])
    set1_snr_array_ch2.insert(x, proc.beacon.snr_by_frame_dB[:,1])
    set1_snr_array_ch3.insert(x, proc.beacon.snr_by_frame_dB[:,2])
    set1_snr_array_ch4.insert(x, proc.beacon.snr_by_frame_dB[:,3])

for x in range(0,file_num2):
    data_samp = "Test" + str(x)
    # filename = data_samp + ".BIN"
    fft_filename = "fflt_" + data_samp + ".bin"
    raw_filename = "r_" + data_samp + ".bin"

    #tee_pos = [1.448, 0.0, 0.1] #driver_pga0_run2/TEST_10.BIN
    #tee_pos = [3.0, 0.0, 0.1]
    #MAKE SURE YOU CHANGE TEE_POS Z (HEIGHT) FOR ELEVATED LM DATA!!
    #tee_pos = [3.0, 0.0, -1]
    # tee_pos = [2.5, 0.5, 2.0]

    proc = golfproc.GolfProcessing(data_dir = data_dir2, \
                               debug_log = 0, debug_plot = 2)

    ##need to choose/set which data file type you have set as 'filename'
    proc.set_system_snapshot(data_dir2, pb_filename=None,
    #                         config_filename="../../../../python/config.json",
                         config_filename=data_dir2 + "/config.json",

                         myconfig_filename="../../../../myconfig.json",
                         fft_filename=fft_filename, waveform_filename=raw_filename) ##uncomment line to use fft data file
    #                         fft_filename=None, waveform_filename=filename) ##uncomment line to use raw data file

    # ri = runtimeinfo.FpgaRuntimeInfo()
    # ri.club_type = 1  # 0=>"Driver", 1=>"5 Iron" currently
    #ri.tee_pos_z = -0.1

    ##PICK ONE and uncomment 
    ##without changing default bins
    aoa_array = proc.do_aoa_processing(0,0*pi/180,1000,15000)
    ##changing default bins
    #proc.do_aoa_processing(0,0,50,100)
    
    set2_mag_array_ch1.insert(x, proc.beacon.magnitude_by_frame_dB[:,0])
    set2_mag_array_ch2.insert(x, proc.beacon.magnitude_by_frame_dB[:,1])
    set2_mag_array_ch3.insert(x, proc.beacon.magnitude_by_frame_dB[:,2])
    set2_mag_array_ch4.insert(x, proc.beacon.magnitude_by_frame_dB[:,3])
    
    set2_noise_array_ch1.insert(x, proc.beacon.noise_by_frame_dB[:,0])
    set2_noise_array_ch2.insert(x, proc.beacon.noise_by_frame_dB[:,1])
    set2_noise_array_ch3.insert(x, proc.beacon.noise_by_frame_dB[:,2])
    set2_noise_array_ch4.insert(x, proc.beacon.noise_by_frame_dB[:,3])
    
    set2_snr_array_ch1.insert(x, proc.beacon.snr_by_frame_dB[:,0])
    set2_snr_array_ch2.insert(x, proc.beacon.snr_by_frame_dB[:,1])
    set2_snr_array_ch3.insert(x, proc.beacon.snr_by_frame_dB[:,2])
    set2_snr_array_ch4.insert(x, proc.beacon.snr_by_frame_dB[:,3])


# SNR Comparison Plots

In [ ]:
set1_matrix_ch1 = np.matrix(set1_snr_array_ch1)
set1_matrix_ch2 = np.matrix(set1_snr_array_ch2)
set1_matrix_ch3 = np.matrix(set1_snr_array_ch3)
set1_matrix_ch4 = np.matrix(set1_snr_array_ch4)

set2_matrix_ch1 = np.matrix(set2_snr_array_ch1)
set2_matrix_ch2 = np.matrix(set2_snr_array_ch2)
set2_matrix_ch3 = np.matrix(set2_snr_array_ch3)
set2_matrix_ch4 = np.matrix(set2_snr_array_ch4)

fig, ax = plt.subplots()

plt.plot(set1_matrix_ch1[0,75:225].mean(0).T, label="ch 1 mean")
plt.plot(set2_matrix_ch1[0,75:225].mean(0).T, label="ch 2 mean")
plt.ylabel('dB')
plt.xlabel('frames')
#plt.ylim(110,125)
avg_set1 = set1_matrix_ch1[0,75:225].mean(1)
avg_set2 = set2_matrix_ch1[0,75:225].mean(1)
avg_diff = avg_set1-avg_set2
plt.title('ch1 snr flight mode comparison' + ' [set 1 mean: ' + str(round(avg_set1[0, 0], 2)) + ' dB]' + '[set 2 mean: ' + str(round(avg_set2[0, 0], 2)) + ' dB]''[diff: ' + str(round(avg_set1[0,0] - avg_set2[0,0],2)) + ' dB]') 
ax.minorticks_on()
# Customize the major grid
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
# Customize the minor grid
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.legend()
plt.savefig("ch1 snr flight mode comparison")

fig, ax = plt.subplots()

plt.plot(set1_matrix_ch2[0,75:225].mean(0).T, label="ch 1 mean")
plt.plot(set2_matrix_ch2[0,75:225].mean(0).T, label="ch 2 mean")
plt.ylabel('dB')
plt.xlabel('frames')
#plt.ylim(110,125)
avg_set1 = set1_matrix_ch2[0,75:225].mean(1)
avg_set2 = set2_matrix_ch2[0,75:225].mean(1)
avg_diff = avg_set1-avg_set2
plt.title('ch2 snr flight mode comparison' + ' (set 1 mean ' + str(round(avg_set1[0, 0], 2)) + 'dB)' + '(set 2 mean ' + str(round(avg_set2[0, 0], 2)) + 'dB)''(diff ' + str(round(avg_set1[0,0] - avg_set2[0,0],2)) + 'dB)') 
ax.minorticks_on()
# Customize the major grid
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
# Customize the minor grid
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.legend()

fig, ax = plt.subplots()

plt.plot(set1_matrix_ch3[0,75:225].mean(0).T, label="ch 1 mean")
plt.plot(set2_matrix_ch3[0,75:225].mean(0).T, label="ch 2 mean")
plt.ylabel('dB')
plt.xlabel('frames')
#plt.ylim(110,125)
avg_set1 = set1_matrix_ch3[0,75:225].mean(1)
avg_set2 = set2_matrix_ch3[0,75:225].mean(1)
avg_diff = avg_set1-avg_set2
plt.title('ch3 snr flight mode comparison' + ' (set 1 mean ' + str(round(avg_set1[0, 0], 2)) + 'dB)' + '(set 2 mean ' + str(round(avg_set2[0, 0], 2)) + 'dB)''(diff ' + str(round(avg_set1[0,0] - avg_set2[0,0],2)) + 'dB)') 
ax.minorticks_on()
# Customize the major grid
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
# Customize the minor grid
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.legend()

fig, ax = plt.subplots()

plt.plot(set1_matrix_ch4[0,75:225].mean(0).T, label="ch 1 mean")
plt.plot(set2_matrix_ch4[0,75:225].mean(0).T, label="ch 2 mean")
plt.ylabel('dB')
plt.xlabel('frames')
#plt.ylim(110,125)
avg_set1 = set1_matrix_ch4[0,75:225].mean(1)
avg_set2 = set2_matrix_ch4[0,75:225].mean(1)
avg_diff = avg_set1-avg_set2
plt.title('ch4 snr flight mode comparison' + ' (set 1 mean ' + str(round(avg_set1[0, 0], 2)) + 'dB)' + '(set 2 mean ' + str(round(avg_set2[0, 0], 2)) + 'dB)''(diff ' + str(round(avg_set1[0,0] - avg_set2[0,0],2)) + 'dB)') 
ax.minorticks_on()
# Customize the major grid
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
# Customize the minor grid
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.legend()

# Magnitude Comparison Plots

In [ ]:
set1_matrix_ch1 = np.matrix(set1_mag_array_ch1)
set1_matrix_ch2 = np.matrix(set1_mag_array_ch2)
set1_matrix_ch3 = np.matrix(set1_mag_array_ch3)
set1_matrix_ch4 = np.matrix(set1_mag_array_ch4)

set2_matrix_ch1 = np.matrix(set2_mag_array_ch1)
set2_matrix_ch2 = np.matrix(set2_mag_array_ch2)
set2_matrix_ch3 = np.matrix(set2_mag_array_ch3)
set2_matrix_ch4 = np.matrix(set2_mag_array_ch4)

fig, ax = plt.subplots()

plt.plot(set1_matrix_ch1[0,75:225].mean(0).T, label="ch 1 mean")
plt.plot(set2_matrix_ch1[0,75:225].mean(0).T, label="ch 2 mean")
plt.ylabel('dB')
plt.xlabel('frames')
plt.ylim(110,125)
avg_set1 = set1_matrix_ch1[0,75:225].mean(1)
avg_set2 = set2_matrix_ch1[0,75:225].mean(1)
avg_diff = avg_set1-avg_set2
plt.title('ch1 mag flight mode comparison' + ' [set 1 mean: ' + str(round(avg_set1[0, 0], 2)) + ' dB]' + '[set 2 mean: ' + str(round(avg_set2[0, 0], 2)) + ' dB]''[diff: ' + str(round(avg_set1[0,0] - avg_set2[0,0],2)) + ' dB]') 
ax.minorticks_on()
# Customize the major grid
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
# Customize the minor grid
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.legend()

fig, ax = plt.subplots()

plt.plot(set1_matrix_ch2[0,75:225].mean(0).T, label="ch 1 mean")
plt.plot(set2_matrix_ch2[0,75:225].mean(0).T, label="ch 2 mean")
plt.ylabel('dB')
plt.xlabel('frames')
plt.ylim(110,125)
avg_set1 = set1_matrix_ch2[0,75:225].mean(1)
avg_set2 = set2_matrix_ch2[0,75:225].mean(1)
avg_diff = avg_set1-avg_set2
plt.title('ch2 mag flight mode comparison' + ' (set 1 mean ' + str(round(avg_set1[0, 0], 2)) + 'dB)' + '(set 2 mean ' + str(round(avg_set2[0, 0], 2)) + 'dB)''(diff ' + str(round(avg_set1[0,0] - avg_set2[0,0],2)) + 'dB)') 
ax.minorticks_on()
# Customize the major grid
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
# Customize the minor grid
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.legend()

fig, ax = plt.subplots()

plt.plot(set1_matrix_ch3[0,75:225].mean(0).T, label="ch 1 mean")
plt.plot(set2_matrix_ch3[0,75:225].mean(0).T, label="ch 2 mean")
plt.ylabel('dB')
plt.xlabel('frames')
plt.ylim(110,125)
avg_set1 = set1_matrix_ch3[0,75:225].mean(1)
avg_set2 = set2_matrix_ch3[0,75:225].mean(1)
avg_diff = avg_set1-avg_set2
plt.title('ch3 mag flight mode comparison' + ' (set 1 mean ' + str(round(avg_set1[0, 0], 2)) + 'dB)' + '(set 2 mean ' + str(round(avg_set2[0, 0], 2)) + 'dB)''(diff ' + str(round(avg_set1[0,0] - avg_set2[0,0],2)) + 'dB)') 
ax.minorticks_on()
# Customize the major grid
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
# Customize the minor grid
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.legend()

fig, ax = plt.subplots()

plt.plot(set1_matrix_ch4[0,75:225].mean(0).T, label="ch 1 mean")
plt.plot(set2_matrix_ch4[0,75:225].mean(0).T, label="ch 2 mean")
plt.ylabel('dB')
plt.xlabel('frames')
plt.ylim(110,125)
avg_set1 = set1_matrix_ch4[0,75:225].mean(1)
avg_set2 = set2_matrix_ch4[0,75:225].mean(1)
avg_diff = avg_set1-avg_set2
plt.title('ch4 mag flight mode comparison' + ' (set 1 mean ' + str(round(avg_set1[0, 0], 2)) + 'dB)' + '(set 2 mean ' + str(round(avg_set2[0, 0], 2)) + 'dB)''(diff ' + str(round(avg_set1[0,0] - avg_set2[0,0],2)) + 'dB)') 
ax.minorticks_on()
# Customize the major grid
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
# Customize the minor grid
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.legend()


# Noise Comparison Plots

In [ ]:
set1_matrix_ch1 = np.matrix(set1_noise_array_ch1)
set1_matrix_ch2 = np.matrix(set1_noise_array_ch2)
set1_matrix_ch3 = np.matrix(set1_noise_array_ch3)
set1_matrix_ch4 = np.matrix(set1_noise_array_ch4)

set2_matrix_ch1 = np.matrix(set2_noise_array_ch1)
set2_matrix_ch2 = np.matrix(set2_noise_array_ch2)
set2_matrix_ch3 = np.matrix(set2_noise_array_ch3)
set2_matrix_ch4 = np.matrix(set2_noise_array_ch4)

fig, ax = plt.subplots()

plt.plot(set1_matrix_ch1[0,75:225].mean(0).T, label="ch 1 mean")
plt.plot(set2_matrix_ch1[0,75:225].mean(0).T, label="ch 2 mean")
plt.ylabel('dB')
plt.xlabel('frames')
#plt.ylim(110,125)
avg_set1 = set1_matrix_ch1[0,75:225].mean(1)
avg_set2 = set2_matrix_ch1[0,75:225].mean(1)
avg_diff = avg_set1-avg_set2
plt.title('ch1 noise flight mode comparison' + ' [set 1 mean: ' + str(round(avg_set1[0, 0], 2)) + ' dB]' + '[set 2 mean: ' + str(round(avg_set2[0, 0], 2)) + ' dB]''[diff: ' + str(round(avg_set1[0,0] - avg_set2[0,0],2)) + ' dB]') 
ax.minorticks_on()
# Customize the major grid
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
# Customize the minor grid
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.legend()

fig, ax = plt.subplots()

plt.plot(set1_matrix_ch2[0,75:225].mean(0).T, label="ch 1 mean")
plt.plot(set2_matrix_ch2[0,75:225].mean(0).T, label="ch 2 mean")
plt.ylabel('dB')
plt.xlabel('frames')
#plt.ylim(110,125)
avg_set1 = set1_matrix_ch2[0,75:225].mean(1)
avg_set2 = set2_matrix_ch2[0,75:225].mean(1)
avg_diff = avg_set1-avg_set2
plt.title('ch2 noise flight mode comparison' + ' (set 1 mean ' + str(round(avg_set1[0, 0], 2)) + 'dB)' + '(set 2 mean ' + str(round(avg_set2[0, 0], 2)) + 'dB)''(diff ' + str(round(avg_set1[0,0] - avg_set2[0,0],2)) + 'dB)') 
ax.minorticks_on()
# Customize the major grid
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
# Customize the minor grid
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.legend()

fig, ax = plt.subplots()

plt.plot(set1_matrix_ch3[0,75:225].mean(0).T, label="ch 1 mean")
plt.plot(set2_matrix_ch3[0,75:225].mean(0).T, label="ch 2 mean")
plt.ylabel('dB')
plt.xlabel('frames')
#plt.ylim(110,125)
avg_set1 = set1_matrix_ch3[0,75:225].mean(1)
avg_set2 = set2_matrix_ch3[0,75:225].mean(1)
avg_diff = avg_set1-avg_set2
plt.title('ch3 noise flight mode comparison' + ' (set 1 mean ' + str(round(avg_set1[0, 0], 2)) + 'dB)' + '(set 2 mean ' + str(round(avg_set2[0, 0], 2)) + 'dB)''(diff ' + str(round(avg_set1[0,0] - avg_set2[0,0],2)) + 'dB)') 
ax.minorticks_on()
# Customize the major grid
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
# Customize the minor grid
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.legend()

fig, ax = plt.subplots()

plt.plot(set1_matrix_ch4[0,75:225].mean(0).T, label="ch 1 mean")
plt.plot(set2_matrix_ch4[0,75:225].mean(0).T, label="ch 2 mean")
plt.ylabel('dB')
plt.xlabel('frames')
#plt.ylim(110,125)
avg_set1 = set1_matrix_ch4[0,75:225].mean(1)
avg_set2 = set2_matrix_ch4[0,75:225].mean(1)
avg_diff = avg_set1-avg_set2
plt.title('ch4 noise flight mode comparison' + ' (set 1 mean ' + str(round(avg_set1[0, 0], 2)) + 'dB)' + '(set 2 mean ' + str(round(avg_set2[0, 0], 2)) + 'dB)''(diff ' + str(round(avg_set1[0,0] - avg_set2[0,0],2)) + 'dB)') 
ax.minorticks_on()
# Customize the major grid
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
# Customize the minor grid
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.legend()